In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np 
import time
import urllib
import re
import os
from random import randint

In [ ]:
# Read in the excel sheet of phone models and counts.
# Remember to change the name of the excel sheet to whatever you saved it as

import datetime
today1 = datetime.date.today()
today = today1.strftime('%b_%Y')

excelfile = 'phone data' + today + '.xlsx'

df = pd.read_excel(excelfile, sheet_name = 'To match')

In [ ]:
# Take the "Models" column
names = df['Model']

In [ ]:
# Preallocate the list of information that we need
listofinfo = []

In [ ]:
# Open browser
browser = webdriver.Chrome()
browser.get("https://google.com")

In [ ]:
# The scraping process
for i in range(0,len(names)):
    list1 = []
    #this searches the term
    searchbar = browser.find_element_by_name("q") # Finding the search bar
    time.sleep(randint(5,33)) 
    searchbar.clear() #clears search bar
    time.sleep(randint(6,28)) 
    searchbar.send_keys('site:gsmarena.com ') 
    searchbar.send_keys(names[i])
    searchbar.send_keys(' specifications')
    searchbar.send_keys(Keys.RETURN)
    time.sleep(randint(5,24))
    
    try: # Try to click the first result if it is there
        
        # This clicks the first result
        browser.find_elements_by_class_name('r')[0].click()
        time.sleep(randint(6,18))
        try: #we try to get the required info if its there
            
            # This gets model name
            modelname = browser.find_element_by_css_selector('h1[data-spec = \'modelname\']')
            list1.append(names[i]) # Appends the name used for search
            list1.append(modelname.text)
        
        except:
            list1.append(names[i]) # Appends the name used for search if information is not there
            list1.append('')
            
        try:
            # This gets size
            dispsize = browser.find_element_by_css_selector('td[data-spec = \'displaysize\']')
            
            # Changed to split by ' ' to get the number only instead of 'number inches'
            list1.append(dispsize.text.split(' ')[0])
        except:
            list1.append('')
            
        try:
            # This gets resolution
            dispreso = browser.find_element_by_css_selector('td[data-spec = \'displayresolution\']')
            list1.append(dispreso.text.split(',')[0])
            
        except:
            list1.append('')
            
        try:
            # This gets OS
            os = browser.find_element_by_css_selector('td[data-spec = \'os\']')
            
            if 'Android' in os.text:
                list1.append('Android')
            elif 'iOS' in os.text:
                list1.append('iOS')
            elif 'Microsoft' in os.text:
                list1.append('Microsoft')
            else:
                list1.append(os.text)
        except:
            list1.append('')
            
        try:
            # This gets price
            price = browser.find_element_by_css_selector('td[data-spec = \'price\']')
            
            # Change to get only the value
            list1.append(price.text.split(' ')[1])
            list1.append('')
            
        except:
            list1.append('')
            list1.append('')
    
    
    except:
        list1.append(names[i])
        list1.append('')
        list1.append('')
        list1.append('')
        list1.append('')
        list1.append('')
        list1.append('phone info not found')
    
    # Put the info of model into the listofinfo
    print(list1)
    listofinfo.append(list1)
    
    browser.back()
    time.sleep(randint(38,147))

In [ ]:
newdf = pd.DataFrame(listofinfo, columns = ['Model', 'Scraped Model', 'Size (Inches)', 'Resolution', 
                                            'OS', 'Price (EUR)', 'Note'])

newdf1 = newdf.set_index('Model')

In [ ]:
newdf1['Size (Inches)'] = pd.to_numeric(newdf1['Size (Inches)'], errors = 'ignore')

In [ ]:
# Scrape brand and device type

# To get brand
newdf1['Brands'] = None

for i in range(0,len(newdf1.index)):
    brand = newdf1.index[i].split()[0]
    newdf1.at[newdf1.index[i],'Brands'] = brand
    
# To get type
newdf1['Type'] = None

for i in range(0,len(newdf1.index)):
    if pd.notna(newdf1.at[newdf1.index[i], 'Size (Inches)']) :
        if newdf1.at[newdf1.index[i], 'Size (Inches)'] >= 7:
            newdf1.at[newdf1.index[i], 'Type'] = 'Tablet'
        else:
            newdf1.at[newdf1.index[i], 'Type'] = 'Phone'
    else:
        newdf1.at[newdf1.index[i], 'Type'] = ''
    

In [ ]:
# Save on new sheet on master list, check manually for errors
from openpyxl import load_workbook

sheetname = 'scraped phone specs'

writer = pd.ExcelWriter(excelfile, engine = 'openpyxl')
writer.book = load_workbook(exclfile)

newdf1.to_excel(writer, sheet_name = sheetname)

writer.save()
writer.close()